# 智能体工具调用循环 - 简化版实现

## 代码作用和目的

这个notebook演示了如何使用**函数调用（Function Calling）**技术来简化AI智能体的实现。相比传统的智能体实现方式，函数调用技术带来了以下重要优势：

### 🎯 核心目的
- **简化智能体循环**：通过函数调用技术，大幅减少解析LLM响应的复杂性
- **结构化响应**：模型原生支持结构化响应，确保返回有效的函数调用或标准文本回复
- **动态工具执行**：智能体可以根据需要动态调用不同的工具函数

### 🔧 技术特点
1. **无需自定义解析逻辑**：不需要设计严格的文本输出解析格式
2. **动态执行**：直接读取和执行模型返回的函数调用
3. **统一处理**：混合处理对话式回复和动作驱动的交互
4. **自动函数映射**：智能体自动将工具名称映射到对应的Python函数

### 📚 学习价值
- 理解函数调用在AI智能体中的应用
- 掌握如何设计工具函数和配置
- 学习智能体循环的基本实现模式
- 了解错误处理和异常管理机制

### 🚀 适用场景
- 文件操作和管理
- 数据查询和处理
- 自动化任务执行
- 智能助手开发

这个实现为初学者提供了一个清晰、易懂的智能体工具调用框架，是学习AI智能体开发的重要起点。


In [ ]:
# 安装必要的依赖包
# litellm是一个统一的LLM接口库，支持多种大语言模型API
!!pip install litellm

# ===========================================
# 环境配置和API密钥设置
# ===========================================

# 重要提示！！！
# 在Google Colab中，需要在左侧面板的"密钥"图标处设置'OPENAI_API_KEY'
# 这是访问OpenAI API的必要步骤
#
import os
from google.colab import userdata

# 从Colab的用户数据中获取OpenAI API密钥
api_key = userdata.get('OPENAI_API_KEY')

# 将API密钥设置为环境变量，供后续代码使用
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
# ===========================================
# 导入必要的库和模块
# ===========================================
import json  # 用于处理JSON数据格式
import os    # 用于操作系统相关操作（如文件列表、文件读取）
from typing import List  # 用于类型提示，提高代码可读性

# 导入litellm库的completion函数，用于调用大语言模型
from litellm import completion

# ===========================================
# 工具函数定义 - 智能体可以调用的具体功能
# ===========================================

def list_files() -> List[str]:
    """
    列出当前目录中的所有文件和文件夹
    
    返回值:
        List[str]: 包含当前目录中所有文件和文件夹名称的列表
    """
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """
    读取指定文件的内容
    
    参数:
        file_name (str): 要读取的文件名
        
    返回值:
        str: 文件内容（如果成功）或错误信息（如果失败）
    """
    try:
        # 尝试以只读模式打开文件
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        # 文件不存在时的错误处理
        return f"Error: {file_name} not found."
    except Exception as e:
        # 其他异常的处理（如权限问题、文件损坏等）
        return f"Error: {str(e)}"

def terminate(message: str) -> None:
    """
    终止智能体循环并显示总结消息
    
    参数:
        message (str): 要显示给用户的终止消息
    """
    print(f"Termination message: {message}")

# ===========================================
# 工具函数映射字典
# ===========================================
# 将工具名称映射到对应的Python函数
# 这样智能体就可以通过工具名称动态调用相应的函数
tool_functions = {
    "list_files": list_files,    # 文件列表工具
    "read_file": read_file,      # 文件读取工具
    "terminate": terminate       # 终止工具
}

# ===========================================
# 工具配置 - 定义智能体可以使用的工具
# ===========================================
# 这个配置告诉大语言模型有哪些工具可以使用
# 以及每个工具的参数要求
tools = [
    {
        "type": "function",  # 工具类型：函数调用
        "function": {
            "name": "list_files",  # 工具名称
            "description": "Returns a list of files in the directory.",  # 工具描述
            "parameters": {"type": "object", "properties": {}, "required": []}  # 参数配置（此工具无需参数）
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_name": {"type": "string"}  # 定义参数类型
                },
                "required": ["file_name"]  # 必需参数列表
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "terminate",
            "description": "Terminates the conversation. No further actions or interactions are possible after this. Prints the provided message for the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {"type": "string"},  # 终止消息参数
                },
                "required": ["message"]  # 消息参数是必需的
            }
        }
    }
]

# ===========================================
# 智能体规则和系统提示
# ===========================================
# 定义智能体的行为规则和角色定位
agent_rules = [{
    "role": "system",  # 系统角色，定义智能体的基本行为
    "content": """
You are an AI agent that can perform tasks by using available tools.

If a user asks about files, documents, or content, first list the files before reading them.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.
"""
}]

# ===========================================
# 智能体循环参数初始化
# ===========================================
iterations = 0        # 当前迭代次数计数器
max_iterations = 10   # 最大迭代次数限制，防止无限循环

# 获取用户任务输入
user_task = input("What would you like me to do? ")

# 初始化对话记忆，存储用户的任务
memory = [{"role": "user", "content": user_task}]

# ===========================================
# 智能体主循环 - 核心执行逻辑
# ===========================================
while iterations < max_iterations:
    
    # 构建完整的消息列表：系统规则 + 对话记忆
    messages = agent_rules + memory

    # 调用大语言模型，获取响应
    # 模型会根据用户任务和可用工具决定下一步行动
    response = completion(
        model="openai/gpt-4o",  # 使用GPT-4o模型
        messages=messages,      # 发送消息历史
        tools=tools,           # 提供可用工具列表
        max_tokens=1024       # 限制响应长度
    )

    # ===========================================
    # 处理工具调用响应
    # ===========================================
    if response.choices[0].message.tool_calls:
        # 如果模型决定调用工具，提取工具信息
        tool = response.choices[0].message.tool_calls[0]
        tool_name = tool.function.name                    # 获取工具名称
        tool_args = json.loads(tool.function.arguments)  # 解析工具参数

        # 构建动作记录，用于记忆管理
        action = {
            "tool_name": tool_name,
            "args": tool_args
        }

        # ===========================================
        # 工具执行逻辑
        # ===========================================
        if tool_name == "terminate":
            # 如果是终止工具，显示消息并退出循环
            print(f"Termination message: {tool_args['message']}")
            break
        elif tool_name in tool_functions:
            # 如果是已知工具，尝试执行
            try:
                # 动态调用对应的工具函数
                result = {"result": tool_functions[tool_name](**tool_args)}
            except Exception as e:
                # 工具执行出错时的错误处理
                result = {"error":f"Error executing {tool_name}: {str(e)}"}
        else:
            # 未知工具的错误处理
            result = {"error": f"Unknown tool: {tool_name}"}

        # 显示执行过程和结果
        print(f"Executing: {tool_name} with args {tool_args}")
        print(f"Result: {result}")
        
        # 将执行结果添加到对话记忆中
        # 这样模型就能记住之前的操作和结果
        memory.extend([
            {"role": "assistant", "content": json.dumps(action)},  # 记录智能体的动作
            {"role": "user", "content": json.dumps(result)}       # 记录执行结果
        ])
    else:
        # 如果模型没有调用工具，直接返回文本响应
        result = response.choices[0].message.content
        print(f"Response: {result}")
        break  # 结束循环


What would you like me to do? read each of the files in the current directory and tell me what they do
Executing: list_files with args {}
Result: {'result': ['.config', 'sample_data']}
Executing: read_file with args {'file_name': '.config'}
Result: {'result': "Error: [Errno 21] Is a directory: '.config'"}
Termination message: It seems that ".config" is a directory, not a file. If you want specific information from within it, please let me know, or ensure there are files inside it to read.
